In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/home/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/home/data/utils.py
/home/data/acs_pca_descriptors_lookup.csv
/home/data/catechol_single_solvent_yields.csv
/home/data/instructions.txt
/home/data/smiles_lookup.csv
/home/data/spange_descriptors_lookup.csv
/home/data/description.md
/home/data/drfps_catechol_lookup.csv
/home/data/catechol_full_data_yields.csv
/home/data/fragprints_lookup.csv


In [2]:
# Define utils functions locally (adapted for local paths)
from typing import Any, Generator

DATA_PATH = '/home/data'

def load_data(name = "full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name = "spange_descriptors"):
    assert name in ["spange_descriptors", "acs_pca_descriptors", "drfps_catechol", "fragprints", "smiles"]
    features = pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)
    return features

def generate_leave_one_out_splits(X, Y):
    all_solvents = X["SOLVENT NAME"].unique()
    for solvent_name in sorted(all_solvents):
        train_idcs_mask = X["SOLVENT NAME"] != solvent_name
        yield (
            (X[train_idcs_mask], Y[train_idcs_mask]),
            (X[~train_idcs_mask], Y[~train_idcs_mask]),
        )

def generate_leave_one_ramp_out_splits(X, Y):
    all_solvent_ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    all_solvent_ramps.sort_values(by=["SOLVENT A NAME", "SOLVENT B NAME"])
    for _, solvent_pair in all_solvent_ramps.iterrows():
        train_idcs_mask = (X[["SOLVENT A NAME", "SOLVENT B NAME"]] != solvent_pair).all(axis=1)
        yield (
            (X[train_idcs_mask], Y[train_idcs_mask]),
            (X[~train_idcs_mask], Y[~train_idcs_mask]),
        )

INPUT_LABELS_FULL_SOLVENT = [
    "Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%",
]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_FEATURES = ["SOLVENT NAME"]
INPUT_LABELS_FULL_FEATURES = ["SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

print('Utils functions defined')

Utils functions defined


In [3]:
from abc import ABC, abstractmethod

class SmilesFeaturizer(ABC):
    def __init__(self):
        raise NotImplementedError

    def featurize(X, Y):
        raise NotImplementedError

class BaseModel(ABC):
    def __init__(self):
        pass

    def train_model(self, X_train, y_train):
        raise NotImplementedError

    def predict(self):
        raise NotImplementedError

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

torch.set_default_dtype(torch.double) 

class PrecomputedFeaturizer(SmilesFeaturizer):
    def __init__(self, features = 'spange_descriptors'):
        assert features in ['drfps_catechol', 'fragprints', 'smiles', 'acs_pca_descriptors', 'spange_descriptors']
        self.features = features
        self.featurizer = load_features(self.features)
        self.feats_dim = self.featurizer.shape[1] + 2

    def featurize(self, X):
        X_numeric = X[INPUT_LABELS_NUMERIC]
        X_smiles = X[INPUT_LABELS_SINGLE_FEATURES]
        X_smiles_feat = self.featurizer.loc[X["SOLVENT NAME"]]
        X_numeric_tensor = torch.tensor(X_numeric.values)
        X_smiles_feat_tensor = torch.tensor(X_smiles_feat.values)
        X_out = torch.cat((X_numeric_tensor, X_smiles_feat_tensor), dim=1)
        return X_out

class PrecomputedFeaturizerMixed(SmilesFeaturizer):
    def __init__(self, features = 'spange_descriptors'):
        assert features in ['drfps_catechol', 'fragprints', 'smiles', 'acs_pca_descriptors', 'spange_descriptors']
        self.features = features
        self.featurizer = load_features(self.features)
        self.feats_dim = self.featurizer.shape[1] + 2

    def featurize(self, X):
        X_numeric = X[INPUT_LABELS_NUMERIC]
        X_smiles = X[INPUT_LABELS_FULL_FEATURES]
        X_smiles_A_feat = self.featurizer.loc[X["SOLVENT A NAME"]]
        X_smiles_B_feat = self.featurizer.loc[X["SOLVENT B NAME"]]
        X_numeric_tensor = torch.tensor(X_numeric.values)
        X_smiles_feat_tensor = X_smiles_A_feat.values * (1 - X["SolventB%"].values.reshape(-1, 1)) + X_smiles_B_feat.values * X["SolventB%"].values.reshape(-1, 1)
        X_smiles_feat_tensor = torch.tensor(X_smiles_feat_tensor)
        X_out = torch.cat((X_numeric_tensor, X_smiles_feat_tensor), dim=1)
        return X_out

class MLPModel(nn.Module, BaseModel):
    def __init__(self, features = 'spange_descriptors', hidden_dims = [64, 64], output_dim = 3, dropout=0.0, data = 'single'):
        super(MLPModel, self).__init__()
        layers = []
        if data == 'single':
            self.smiles_featurizer = PrecomputedFeaturizer(features = features)
        else:
            self.smiles_featurizer = PrecomputedFeaturizerMixed(features = features)
        input_dim = self.smiles_featurizer.feats_dim
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.ReLU())
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.model = nn.Sequential(*layers)

    def train_model(self, train_X, train_Y, criterion=nn.MSELoss, optimizer=torch.optim.Adam,
                    num_epochs=100, batch_size=1048, device="cpu", verbose=True, lr=1e-3):
        self.train()
        train_X_tensor = self.smiles_featurizer.featurize(train_X)
        train_Y_tensor = torch.tensor(train_Y.values)
        train_dataset = TensorDataset(train_X_tensor, train_Y_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        if device is None:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(device)
        criterion = criterion()
        optimizer = optimizer(self.parameters(), lr=lr)
        for epoch in range(1, num_epochs + 1):
            self.train()
            running_loss = 0.0
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * inputs.size(0)
            epoch_loss = running_loss / len(train_loader.dataset)

    def predict(self, X):
        X = self.smiles_featurizer.featurize(X)
        return self.forward(X)

    def forward(self, x):
        return self.model(x)

print('MLPModel defined')

MLPModel defined


In [5]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MLPModel() # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:01,  1.04s/it]

2it [00:01,  1.57it/s]

3it [00:01,  2.10it/s]

4it [00:02,  2.33it/s]

...

21it [00:07,  2.91it/s]

22it [00:08,  2.99it/s]

23it [00:08,  3.05it/s]

24it [00:08,  2.85it/s]

24it [00:08,  2.73it/s]

In [6]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MLPModel(data = 'full') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:00,  1.78it/s]

2it [00:01,  1.63it/s]

3it [00:01,  1.63it/s]

4it [00:02,  1.75it/s]

...

10it [00:05,  1.71it/s]

11it [00:06,  1.66it/s]

12it [00:07,  1.61it/s]

13it [00:07,  1.59it/s]

13it [00:07,  1.66it/s]

In [7]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# CV calculation (for local verification only - NOT part of submission)\nfrom sklearn.metrics import mean_squared_error\n\n# Single solvent CV\nX_single, Y_single = load_data(\"single_solvent\")\nsplit_gen = list(generate_leave_one_out_splits(X_single, Y_single))\nall_y_true, all_y_pred = [], []\nfor fold_idx, split in enumerate(split_gen):\n    (_, _), (_, test_Y) = split\n    fold_preds = submission_single_solvent[submission_single_solvent['fold'] == fold_idx]\n    all_y_true.append(test_Y.values)\n    all_y_pred.append(fold_preds[['target_1', 'target_2', 'target_3']].values)\nmse_single = mean_squared_error(np.vstack(all_y_true), np.vstack(all_y_pred))\n\n# Full data CV\nX_full, Y_full = load_data(\"full\")\nsplit_gen = list(generate_leave_one_ramp_out_splits(X_full, Y_full))\nall_y_true, all_y_pred = [], []\nfor fold_idx, split in enumerate(split_gen):\n    (_, _), (_, test_Y) = split\n    fold_preds = submission_full_data[submission_full_data['fold'] == fold_idx]\n    all_y_true.append(test_Y.values)\n    all_y_pred.append(fold_preds[['target_1', 'target_2', 'target_3']].values)\nmse_full = mean_squared_error(np.vstack(all_y_true), np.vstack(all_y_pred))\n\nprint(f'Single Solvent CV MSE: {mse_single:.6f}')\nprint(f'Full Data CV MSE: {mse_full:.6f}')